# SOC LEAFs Forest and Grassland supporting calculations
This notebook documents the generation of forest and grassland base data needed to run RothC model and serves as a complement to [SOC_LEAFs_Baseline](../documentation/SOC_LEAFs_Baseline.ipynb) notebook.

Notes from IPCC 2003
- Litter to carbon: To convert to dry matter mass of litter to carbon, multiply the mass by a default value of 0.370 (Smith and Heath, 2002), not the carbon fraction used for biomass.
- Transition time: The transition from Ci to Cj is assumed to take place over a transition period of T years (default = 20 years).